### Libs 

In [64]:
import pandas as pd
import openpyxl as xl
import schedule 
import time 
import numpy as np
from datetime import datetime,timedelta
from IPython.display import display

### Leitura Dataseat

In [65]:
def leitura_excel():
    try:
        # Lê o arquivo CSV
        df = pd.read_csv(r'C:\Users\gm194\Downloads\despesas_contratadas_candidatos_2024_RS.csv', encoding="latin-1", sep=";", nrows=2000)
        display(df.head(1))  # Imprime as primeiras linhas do DataFrame
        return df  # Retorna o DataFrame se a leitura for bem-sucedida
    except Exception as e:
        print(f"Erro ao ler o arquivo: {e}")  # Tratamento de erros
        return None  # Retorna None se houver um erro

# Executa a função e armazena o DataFrame em uma variável, para poder usar em outras células
df = leitura_excel()

# Verifica se a leitura foi bem-sucedida e exibe o DataFrame completo
if df is not None:
    print("A leitura foi concluída com sucesso.")
else:
    print("A leitura falhou.")

,DT_GERACAO,HH_GERACAO,AA_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,ST_TURNO,TP_PRESTACAO_CONTAS,...,SG_PARTIDO_FORNECEDOR,NM_PARTIDO_FORNECEDOR,DS_TIPO_DOCUMENTO,NR_DOCUMENTO,CD_ORIGEM_DESPESA,DS_ORIGEM_DESPESA,SQ_DESPESA,DT_DESPESA,DS_DESPESA,VR_DESPESA_CONTRATADA
0,26/10/2024,03:00:57,2024,2,Ordinária,619,Eleições Municipais 2024,06/10/2024,1,Parcial,...,#NULO,#NULO,Nota Fiscal,820,20110000,Publicidade por adesivos,54764890,09/09/2024,ADESIVO MICROPERFURADO,"120,00"


A leitura foi concluída com sucesso.


### Manipulação de dados

In [72]:
# Converte strings para datas. Valores inválidos se tornam NaT( Not a time).
# O parâmetro erros='coerce' é usado para evitar erros na conversão
# Se a data não for válida, ela é transformada em NaT em vez de parar o código.
df['DT_GERACAO'] = pd.to_datetime(df['DT_GERACAO'], errors='coerce', dayfirst=False)
df['DT_ELEICAO'] = pd.to_datetime(df['DT_ELEICAO'], errors='coerce',dayfirst=False)
df['DT_DESPESA'] = pd.to_datetime(df['DT_DESPESA'], errors='coerce',dayfirst=False)

#Após tratar essa coluna, posso passar para string e mudar o formato da data
df['DT_GERACAO'] = df['DT_GERACAO'].dt.strftime('%Y-%m-%d')
df['DT_ELEICAO'] = df['DT_ELEICAO'].dt.strftime('%Y-%m-%d')
df['DT_DESPESA'] = df['DT_DESPESA'].dt.strftime('%Y-%m-%d')

## Tratando valores nulos

In [73]:
df['SG_PARTIDO_FORNECEDOR'] = df['SG_PARTIDO_FORNECEDOR'].replace({'#NULO': np.nan})
df['NM_PARTIDO_FORNECEDOR'] = df['NM_PARTIDO_FORNECEDOR'].replace({'#NULO': np.nan})

df.head(1)


,DT_GERACAO,HH_GERACAO,AA_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,ST_TURNO,TP_PRESTACAO_CONTAS,...,SG_PARTIDO_FORNECEDOR,NM_PARTIDO_FORNECEDOR,DS_TIPO_DOCUMENTO,NR_DOCUMENTO,CD_ORIGEM_DESPESA,DS_ORIGEM_DESPESA,SQ_DESPESA,DT_DESPESA,DS_DESPESA,VR_DESPESA_CONTRATADA
0,2024-10-26,03:00:57,2024,2,Ordinária,619,Eleições Municipais 2024,2024-10-06,1,Parcial,...,NaN,NaN,Nota Fiscal,820,20110000,Publicidade por adesivos,54764890,2024-09-09,ADESIVO MICROPERFURADO,"120,00"
